In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

# 라이브러리 적용

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from PIL import Image

import numpy as np

import copy
import pickle
import os

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

# 훈련된 모델 받기

In [ ]:
os.path.isdir('stylegan2-ada-pytorch')

In [ ]:
if not os.path.isdir('stylegan2-ada-pytorch'):
  !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

%cd stylegan2-ada-pytorch

In [ ]:
if not os.path.isdir('pretrained'):
  !mkdir pretrained
  %cd pretrained
  !wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
  %cd ..

In [ ]:
with open('pretrained/ffhq.pkl', 'rb') as f:
  G = pickle.load(f)['G_ema'].cuda()


# 그림 생성하기

In [ ]:
z = torch.randn((1, G.z_dim)).cuda()
c = None

img = G(z, c)

## 그림을 numpy의 배열로 변환

In [ ]:
target_unit8 = (img[0]* 127.5 + 128).clamp(0,255).detach().permute(1,2,0).cpu().to(torch.uint8)

In [ ]:
plt.imshow(target_uint8)
plt.axis('off')
plt.show()

## 그림 정렬, 정돈
### Double-cliquez pour modifier


In [ ]:
!mkdir -p raw
!wget https://upload.wikimedia.org/wikipedia/commons/0/0e/Donald_Trump_Pentagon_2017.jpg -0 raw/example.jpg

그림에서 얼굴을 뽑아내고, 정렬하기 - DLib와 그 함수를 이용하여 원본 FFHQ 데이터셋의 준비과정을 거친 부분에서

In [ ]:
!python align_images.py raw aligned

샘플로 만든 그림을 이미 훈련된 모델의 latent space로 변환

In [ ]:
!ls

In [ ]:
sample_flickr_dog_path = os.path.join('aligned', 'example_01.png')
target_img = Image.open(sample_flickr_dog_path)
target_img

In [ ]:
target_uint8 = np.array(target_img, dtype=np.uint8)


이미 훈련된 Generator 불러오기

In [ ]:
device = torch.device('cuda')
G_eval = copy.deepcopy(G).eval().requires_grad_(False).to(device) # 모델을 평가 형태로 쓰기!!!

w stats 계산

In [ ]:
z_samples = np.random.randn(10000, G_eval.z_dim)
w_samples = G_eval.mapping(torch.from_numpy(z_samples).to(device), None)
w_samples.size()

In [ ]:
w_samples = w_samples[:, :1, :].cpu().numpy().astype(np.float32)
w_samples

In [ ]:
w_samples.shape

In [ ]:
w_avg = np.mean(w_samples, axis=0, keepdims=True)
w_avg.shape

In [ ]:
w_std = (np.sum((w_samples - w_avg) ** 2) / w_samples.shape[0]) ** 0.5
w_std

noise 입력 설정

In [ ]:
noise_bufs = {name: buf for (name, buf) in G_eval.synthesis.named_buffers() if 'noise_const' in name}

VGG16의 특성 감지 불러오기

In [ ]:
url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metrics/vgg16.pt'
with dnnlib.util.open_url(url) as f:
  vgg16 = torch.jit.load(f).eval().to(device)

그림에서 특성 추출

In [ ]:
target = torch.tensor(target_uint8, device=device).permute(2,0,1)
target.size()

In [ ]:
target = target.unsqueeze(0).to(device).to(torch.float32)
target.size()

In [ ]:
target = F.interpolate(target, size=(256, 256), mode='area') # VGG16의 크기에 맞게 변경
target.size()

In [ ]:
target_features = vgg16(target, resize_images=False, return_lpips=True)
target_features

In [ ]:
target_features.size()

최적화 설정,

In [ ]:
num_steps = 1000
initial_learning_rate = 0.1

w_opt = torch.tensor(w_avg, dtype=torch.float32, device=device, requires_grad=True)
w_out = torch.zeros([num_steps] + list(w_opt.shape[1:]), dtype=torch.float32, device=device)
optimizer = torch.optim.Adam([w_opt] + list(noise_bufs.values()), betas=(0.9, 0.999), lr=initial_learning_rate)

# 노이즈 초기화
for buf in noise_bufs.values():
  buf[:] = torch.randn_like(buf)
  buf.requires_grad = True

projection

loss를 줄이기 위해 ....


In [ ]:
num_steps = 1000
# 공식적으로 제공되는 기본 하이퍼 파라미터 값으로 지정
lr_rampdown_length = 0.25
lr_rampup_length = 0.05
initial_noise_factor = 0.05
noise_ramp_length = 0.75
regularize_noise_weight = 1e5

#
best_loss = np.inf

for step in tqdm(range(num_steps)):
  # Learning rate를 적절하게 조절
  t = step / num_steps
  w_noise_scale = w_std * initial_noise_factor * max(0.0, 1.0 - t / noise_ramp_length) ** 2
  lr_ramp = min(1.0, (1.0 - t) / lr_rampdown_length)
  lr_ramp = 0.5 -0.5 * np.cos(lr_ramp * np.pi)
  lr_ramp = lr_ramp * min(1.0, t / lr_rampup_length)
  lr = initial_learning_rate * lr_ramp
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr
  # 그림 생성
  w_noise = torch.randn_like(w_opt) * w_noise_scale
  ws = (w_opt + w_noise).repeat([1, G_eval.synthesis.num_ws, 1])
  synth_images = G.synthesis(ws, noise_mode='const')

  # 그림을 256x256 크기로 줄이기 VGG는 224x224의 그림에 맞게 설정되었음
  synth_images = (synth_images + 1) * (255/2)
  if synth_images.shape[2] > 256:
    synth_images = F.interpolate(synth_images, size=(256, 256), mode='area')

  # 생성된 그림의 특성들
  synth_features = vgg16(synth_images, resize_images=False, return_lpips=True)
  dist = (target_features - synth_features).square().sum() # 두 feature maps간의 차이를 계산 ( target, synth) > projection의 point

  # noise 재정규화
  reg_loss = 0.0
  for v in noise_bufs.values():
    noise = v[None,None,:,:] # F.avg_pool2d()에 쓰이려면 (1,1,H,W) 여야 함
    while True:
      reg_loss += (noise * torch.roll(noise, shifts=1, dims=3)).mean() ** 2
      reg_loss += (noise * torch.roll(noise, shifts=1, dims=2)).mean() ** 2
      if noise.shape[2] <= 8:
        break
      noise = F.avg_pool2d(noise, kernel_size=2)
  loss = dist + reg_loss * regularize_noise_weight
  # Step!!!
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(f'step {step+1:>4d}/{num_steps}: dist {dist:<4.2f} loss {float(loss):<5.2f}')

  # 각 optimization step마다 사영된 w를 저장
  w_out[step] = w_opt.detach()[0]

  # noise를 normalize
  with torch.no_grad():
    for buf in noise_bufs.values():
      buf -= buf.mean()
      buf *= buf.square().mean().rsqrt()


In [ ]:
w_out.size()

In [ ]:
w_out.repeat([1, G.mapping.num_w5, 1]).size()

In [ ]:
projected_w_steps = w_out.repeat([1, G.mapping.num_ws, 1])
projected_w = projected_w_steps[-1]
print(projected_w.size())
print(projected_w)

In [ ]:
np_w = projected_w.cpu().numpy()
np.save('./projected_w', np_w)

In [ ]:
synth_image = G.syntehsis(projected_w.unsqueeze(0), noise_mode='const')
synth_image = (synth_image + 1) * (255/2)
synth_image = synth_image.permute(0,2,3,1).clamp(0,255).to(torch.uint8)[0].cpu().numpy()

In [ ]:
plt.imshow(np.concatenate([target_uint8, synth_image], axis=1))
plt.axis('off')
plt.title('synth <-- vs --> target')
plt.show()